## Before starting run cells and restart runtime

In [ ]:
pip install plotly-geo

In [ ]:
pip install geopandas

     |████████████████████████████████| 931kB 3.5MB/s 
     |████████████████████████████████| 14.7MB 304kB/s 
     |████████████████████████████████| 10.4MB 44.7MB/s 


In [ ]:
pip install pyshp

In [ ]:
pip install shapely

## Loading Data, First Glance

In [ ]:
import pandas as pd

In [ ]:
!wget -q -O data2.csv https://www.dropbox.com/s/yssj6fysckdf84g/mldata_2.csv?dl=0
data_2 = pd.read_csv('data2.csv')

!wget -q -O data3.csv https://www.dropbox.com/s/0qfcxu8uigfv9oo/more_ml_data.csv?dl=0
data_3 = pd.read_csv('data3.csv')

!wget -q -O gender.csv https://www.dropbox.com/s/rqsl7845wevp5sg/ml_gender_year.csv?dl=0
gender_data = pd.read_csv('gender.csv')

!wget -q -O race.csv https://www.dropbox.com/s/dwauuv8q7p7o0my/ml_race_year.csv?dl=0
race_data = pd.read_csv('race.csv')

!wget -q -O race.csv https://www.dropbox.com/s/zn2pfs9b45haqcr/ml_urban.csv?dl=0
urban_data = pd.read_csv('race.csv')


In [ ]:
data_2.describe()

,fips,trump16,clinton16,otherpres16,romney12,obama12,otherpres12,demhouse16,rephouse16,otherhouse16,demgov16,total_population,cvap,white_pct,black_pct,hispanic_pct,nonwhite_pct,foreignborn_pct,female_pct,age29andunder_pct,age65andolder_pct,median_hh_inc,clf_unemploy_pct,lesshs_pct,lesscollege_pct,lesshs_whites_pct,lesscollege_whites_pct,rural_pct,ruralurban_cc
count,134.000000,134.000000,134.000000,134.000000,134.000000,134.000000,134.000000,101.000000,101.000000,101.000000,0.0,1.330000e+02,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,134.000000,134.000000
mean,51265.865672,13204.798507,14787.111940,1744.141791,13600.910448,14715.074627,448.858209,12625.811881,13990.316832,704.277228,NaN,6.248347e+04,45104.511278,71.972572,18.552613,4.941769,28.027428,5.249693,50.615785,36.473420,17.644459,54078.413534,6.622902,14.826893,74.471462,11.952205,71.317980,51.989520,3.708955
std,271.725558,20202.601267,36860.963337,4045.649986,23653.714885,33306.150667,795.733134,38057.644357,18552.763487,3666.384064,NaN,1.235467e+05,81133.985957,18.217012,16.473288,5.626631,18.217012,6.377846,2.938385,7.049176,5.069728,19723.498829,2.499767,5.494440,13.207827,5.609799,15.141906,40.446908,2.798562
min,51001.000000,0.000000,0.000000,0.000000,886.000000,459.000000,17.000000,324.000000,823.000000,0.000000,NaN,2.230000e+03,1920.000000,14.988905,0.000000,0.000000,0.448430,0.019249,34.245291,20.538117,6.653188,26000.000000,1.306533,2.047599,19.789880,0.576752,14.167406,0.000000,1.000000
25%,51069.500000,3559.750000,2307.500000,245.000000,3575.500000,2742.000000,95.000000,2210.000000,4172.000000,20.000000,NaN,1.483100e+04,10960.000000,60.072524,5.288084,1.839303,12.511358,1.805007,50.177804,32.307013,14.732432,39911.000000,5.006247,10.515464,69.579225,7.826758,65.674012,3.743624,1.000000
50%,51140.000000,7086.000000,4187.000000,512.500000,6449.500000,4718.000000,191.000000,3385.000000,7760.000000,78.000000,NaN,2.529000e+04,19965.000000,74.281959,13.006954,3.058071,25.718041,2.974809,50.908488,34.880853,18.166185,49639.000000,6.343384,14.845241,77.449308,11.256893,74.796263,60.917346,3.000000
75%,51547.500000,14615.250000,9005.000000,1249.500000,13948.750000,9911.500000,476.250000,6493.000000,14323.000000,295.000000,NaN,5.456200e+04,42760.000000,87.488642,28.877798,5.968046,39.927476,6.003452,51.800231,38.991696,20.194827,64049.000000,7.704836,18.842335,83.911151,15.543951,81.997580,97.106284,6.000000
max,51840.000000,157710.000000,355133.000000,38340.000000,206773.000000,315273.000000,7241.000000,339485.000000,103982.000000,35619.000000,NaN,1.132887e+06,715975.000000,99.551570,76.547801,36.526189,85.011095,33.234346,56.418468,67.367823,36.053812,125672.000000,18.771186,29.972847,91.501155,30.024754,90.442553,100.000000,9.000000


In [ ]:
#Feature engineering -- what we will predict?
data_2["%_Repub16"]=data_2["trump16"]/(data_2["trump16"]+ data_2["otherpres16"]+data_2["clinton16"]) 
data_2["%_Dem16"]=data_2["clinton16"]/(data_2["trump16"]+ data_2["otherpres16"]+data_2["clinton16"]) 
data_2["%_Repub12"]=data_2["romney12"]/(data_2["romney12"]+ data_2["otherpres12"]+data_2["obama12"]) 
data_2["%_Dem12"]=data_2["obama12"]/(data_2["romney12"]+ data_2["otherpres12"]+data_2["obama12"]) 

## Visualizations/Exploration

In [ ]:
import plotly
import plotly.figure_factory as ff
from plotly.offline import *
import plotly.express as px


fig = ff.create_choropleth(
	  fips = data_2['fips'], values = data_2["%_Repub16"],
	  show_state_data = True, scope = ['VA'],
	  county_outline={'color': 'rgb(15,15,55)', 'width': 1},
	  state_outline={'color': 'rgb(15,15,55)', 'width': 1},
	  legend_title='Percent for Trump', 
    title='Percent of Voters who voted for Trump in 2016',
)

plotly.offline.iplot(fig)

fig = ff.create_choropleth(
	  fips = data_2['fips'], values = data_2["%_Dem16"],
	  show_state_data = True, scope = ['VA'],
	  county_outline={'color': 'rgb(15,15,55)', 'width': 1},
	  state_outline={'color': 'rgb(15,15,55)', 'width': 1},
	  legend_title='Percent for Clinton', 
    title='Percent of Voters who voted for Clinton in 2016',
	  )

plotly.offline.iplot(fig)

In [ ]:
democrat = data_3[data_3['party'] == 'democrat'].drop(columns=['office', 'candidate', 'party'])
republican = data_3[data_3['party'] == 'republican'].drop(columns=['office', 'candidate', 'party'])
mask = data_3['party'] != 'democrat'
mask = mask & (data_3['party'] != 'republican')
others = data_3[mask].drop(columns=['office', 'candidate', 'party'])

In [ ]:
print(data_3.shape)
print(democrat.shape)
print(republican.shape)
print(others.shape)

other = (others.groupby(['year','FIPS']).sum()).reset_index()

(2144, 8)
(670, 5)
(670, 5)
(804, 5)


#Data Discovery, Cleaning, and Scaling

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedShuffleSplit


In [ ]:
democrat_nan = democrat.dropna(subset=["candidatevotes"])
republican_nan = republican.dropna(subset=["candidatevotes"])

In [ ]:
democrat_X_iter = democrat_nan.copy()
republican_X_iter = republican_nan.copy()

helper methods

In [ ]:
def get_overlapping_stats(overlap_fips, fips, candidate, total):
  ret_fips = []
  ret_candidate = []
  ret_total = []
  for fip in fips:
    try:
      index = overlap_fips.index(fip)
      ret_fips.append(fip)
      ret_candidate.append(candidate[index])
      ret_total.append(total[index])
    except:
      # do nothing
      x = 4
  return ret_fips, ret_candidate, ret_total
  
def sort_by_fips(fips, candidate, total):
  return zip(*sorted(zip(fips, candidate, total)))

# Splitting into Train/Test and Building Models

In [ ]:
import sys
import sklearn
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from functools import partial
import tensorflow as tf

%tensorflow_version 2.x


np.random.seed(49)
tf.random.set_seed(49)


# Train/Test

In [ ]:
X_train = pd.DataFrame()
y_train_democrat = pd.DataFrame()
y_train_republican = pd.DataFrame()

X_test = pd.DataFrame()
y_test_democrat = pd.DataFrame()
y_test_republican = pd.DataFrame()

## Democrat Training/Testing Sets

In [ ]:
OO_candidate = []
OO_total = []
OO_fips = []


O4_candidate = []
O4_total = []
O4_fips = []

O8_candidate = []
O8_total = []
O8_fips = []

O12_candidate = []
O12_total = []
O12_fips = []

O16_candidate = []
O16_total = []
O16_fips = []


for index, row in democrat_X_iter.iterrows():
    if row['year'] == 2000:
      OO_fips.append(row['FIPS'])
      OO_candidate.append(row['candidatevotes'])
      OO_total.append(row['totalvotes'])
    elif row['year'] == 2004:
      O4_fips.append(row['FIPS'])
      O4_candidate.append(row['candidatevotes'])
      O4_total.append(row['totalvotes'])
    elif row['year'] == 2008:
      O8_fips.append(row['FIPS'])
      O8_candidate.append(row['candidatevotes'])
      O8_total.append(row['totalvotes'])
    elif row['year'] == 2012:
      O12_fips.append(row['FIPS'])
      O12_candidate.append(row['candidatevotes'])
      O12_total.append(row['totalvotes'])
    else:
      O16_fips.append(row['FIPS'])
      O16_candidate.append(row['candidatevotes'])
      O16_total.append(row['totalvotes'])

all_fips = set(OO_fips.copy())
all_fips = list(all_fips.intersection(*[O4_fips, O8_fips, O12_fips, O16_fips]))



OO_fips, OO_candidate, OO_total = get_overlapping_stats(all_fips, OO_fips, OO_candidate, OO_total)
O4_fips, O4_candidate, O4_total = get_overlapping_stats(all_fips, O4_fips, O4_candidate, O4_total)
O8_fips, O8_candidate, O8_total = get_overlapping_stats(all_fips, O8_fips, O8_candidate, O8_total)
O12_fips, O12_candidate, O12_total = get_overlapping_stats(all_fips, O12_fips, O12_candidate, O12_total)
O16_fips, O16_candidate, O16_total = get_overlapping_stats(all_fips, O16_fips, O16_candidate, O16_total)



all_fips = sorted(all_fips)
OO_fips, OO_candidate, OO_total = sort_by_fips(OO_fips, OO_candidate, OO_total)
O4_fips, O4_candidate, O4_total = sort_by_fips(O4_fips, O4_candidate, O4_total)
O8_fips, O8_candidate, O8_total = sort_by_fips(O8_fips, O8_candidate, O8_total)
O12_fips, O12_candidate, O12_total = sort_by_fips(O12_fips, O12_candidate, O12_total)
O16_fips, O16_candidate, O16_total = sort_by_fips(O16_fips, O16_candidate, O16_total)


In [ ]:
democrat_X = pd.DataFrame()

democrat_X['fips'] = all_fips

democrat_X['1_candidate_dem'] = O8_candidate
democrat_X['1_total_dem'] = O8_total
democrat_X['1_percent_dem'] = democrat_X['1_candidate_dem']/democrat_X['1_total_dem']


democrat_X['2_candidate_dem'] = O4_candidate
democrat_X['2_total_dem'] = O4_total
democrat_X['2_percent_dem'] = democrat_X['2_candidate_dem']/democrat_X['2_total_dem']

democrat_X['3_candidate_dem'] = OO_candidate
democrat_X['3_total_dem'] = OO_total
democrat_X['3_percent_dem'] = democrat_X['3_candidate_dem']/democrat_X['3_total_dem']

y_train_democrat['candidate'] = O12_candidate
y_train_democrat['total'] = O12_total
y_train_democrat['percent'] = y_train_democrat['candidate'] / y_train_democrat['total']
y_train_democrat.drop(columns=['candidate', 'total'], inplace=True)


########################################################################################3

democrat_X_test = pd.DataFrame()

democrat_X_test['fips'] = all_fips

democrat_X_test['1_candidate_dem'] = O12_candidate
democrat_X_test['1_total_dem'] = O12_total
democrat_X_test['1_percent_dem'] = democrat_X_test['1_candidate_dem']/democrat_X_test['1_total_dem']

democrat_X_test['2_candidate_dem'] = O8_candidate
democrat_X_test['2_total_dem'] = O8_total
democrat_X_test['2_percent_dem'] = democrat_X_test['2_candidate_dem']/democrat_X_test['2_total_dem']


democrat_X_test['3_candidate_dem'] = O4_candidate
democrat_X_test['3_total_dem'] = O4_total
democrat_X_test['3_percent_dem'] = democrat_X_test['3_candidate_dem']/democrat_X_test['3_total_dem']


y_test_democrat = pd.DataFrame()
y_test_democrat['candidate'] = O16_candidate
y_test_democrat['total'] = O16_total
y_test_democrat['percent'] = y_test_democrat['candidate'] / y_test_democrat['total']
y_test_democrat.drop(columns=['candidate', 'total'], inplace=True)

########################################################################################

dem_2020 = pd.DataFrame()

dem_2020['fips'] = all_fips
dem_2020['1_candidate_dem'] = O16_candidate
dem_2020['1_total_dem'] = O16_total
dem_2020['1_percent_dem'] = dem_2020['1_candidate_dem']/dem_2020['1_total_dem']

dem_2020['2_candidate_dem'] = O12_candidate
dem_2020['2_total_dem'] = O12_total
dem_2020['2_percent_dem'] = dem_2020['2_candidate_dem']/dem_2020['2_total_dem']


dem_2020['3_candidate_dem'] = O8_candidate
dem_2020['3_total_dem'] = O8_total
dem_2020['3_percent_dem'] = dem_2020['3_candidate_dem']/dem_2020['3_total_dem']

## Republican Training/Testing Sets

In [ ]:
republican_X = pd.DataFrame()

OO_candidate = []
OO_total = []
OO_fips = []

O4_candidate = []
O4_total = []
O4_fips = []

O8_candidate = []
O8_total = []
O8_fips = []

O12_candidate = []
O12_total = []
O12_fips = []

O16_candidate = []
O16_total = []
O16_fips = []

for index, row in republican_X_iter.iterrows():
    if row['year'] == 2000:
      OO_fips.append(row['FIPS'])
      OO_candidate.append(row['candidatevotes'])
      OO_total.append(row['totalvotes'])
    elif row['year'] == 2004:
      O4_fips.append(row['FIPS'])
      O4_candidate.append(row['candidatevotes'])
      O4_total.append(row['totalvotes'])
    elif row['year'] == 2008:
      O8_fips.append(row['FIPS'])
      O8_candidate.append(row['candidatevotes'])
      O8_total.append(row['totalvotes'])
    elif row['year'] == 2012:
      O12_fips.append(row['FIPS'])
      O12_candidate.append(row['candidatevotes'])
      O12_total.append(row['totalvotes'])
    else:
      O16_fips.append(row['FIPS'])
      O16_candidate.append(row['candidatevotes'])
      O16_total.append(row['totalvotes'])

all_fips = set(OO_fips.copy())
all_fips = list(all_fips.intersection(*[O4_fips, O8_fips, O12_fips, O16_fips]))


OO_fips, OO_candidate, OO_total = get_overlapping_stats(all_fips, OO_fips, OO_candidate, OO_total)
O4_fips, O4_candidate, O4_total = get_overlapping_stats(all_fips, O4_fips, O4_candidate, O4_total)
O8_fips, O8_candidate, O8_total = get_overlapping_stats(all_fips, O8_fips, O8_candidate, O8_total)
O12_fips, O12_candidate, O12_total = get_overlapping_stats(all_fips, O12_fips, O12_candidate, O12_total)
O16_fips, O16_candidate, O16_total = get_overlapping_stats(all_fips, O16_fips, O16_candidate, O16_total)


all_fips = sorted(all_fips)
OO_fips, OO_candidate, OO_total = sort_by_fips(OO_fips, OO_candidate, OO_total)
O4_fips, O4_candidate, O4_total = sort_by_fips(O4_fips, O4_candidate, O4_total)
O8_fips, O8_candidate, O8_total = sort_by_fips(O8_fips, O8_candidate, O8_total)
O12_fips, O12_candidate, O12_total = sort_by_fips(O12_fips, O12_candidate, O12_total)
O16_fips, O16_candidate, O16_total = sort_by_fips(O16_fips, O16_candidate, O16_total)




In [ ]:
republican_X = pd.DataFrame()

republican_X['fips'] = all_fips

republican_X['1_candidate_rep'] = O8_candidate
republican_X['1_total_rep'] = O8_total
republican_X['1_percent_rep'] = republican_X['1_candidate_rep']/republican_X['1_total_rep']

republican_X['2_candidate_rep'] = O4_candidate
republican_X['2_total_rep'] = O4_total
republican_X['2_percent_rep'] = republican_X['2_candidate_rep']/republican_X['2_total_rep']

republican_X['3_candidate_rep'] = OO_candidate
republican_X['3_total_rep'] = OO_total
republican_X['3_percent_rep'] = republican_X['3_candidate_rep']/republican_X['3_total_rep']

y_train_republican['candidate'] = O12_candidate
y_train_republican['total'] = O12_total
y_train_republican['percent'] = y_train_republican['candidate'] / y_train_republican['total']
y_train_republican.drop(columns=['candidate', 'total'], inplace=True)

########################################################################################3

republican_X_test = pd.DataFrame()

republican_X_test['fips'] = all_fips

republican_X_test['1_candidate_rep'] = O12_candidate
republican_X_test['1_total_rep'] = O12_total
republican_X_test['1_percent_rep'] = republican_X_test['1_candidate_rep']/republican_X_test['1_total_rep']

republican_X_test['2_candidate_rep'] = O8_candidate
republican_X_test['2_total_rep'] = O8_total
republican_X_test['2_percent_rep'] = republican_X_test['2_candidate_rep']/republican_X_test['2_total_rep']

republican_X_test['3_candidate_rep'] = O4_candidate
republican_X_test['3_total_rep'] = O4_total
republican_X_test['3_percent_rep'] = republican_X_test['3_candidate_rep']/republican_X_test['3_total_rep']

y_test_republican = pd.DataFrame()
y_test_republican['candidate'] = O16_candidate
y_test_republican['total'] = O16_total
y_test_republican['percent'] = y_test_republican['candidate'] / y_test_republican['total']
y_test_republican.drop(columns=['candidate', 'total'], inplace=True)

########################################################################################3

rep_2020 = pd.DataFrame()

rep_2020['fips'] = all_fips

rep_2020['1_candidate_rep'] = O16_candidate
rep_2020['1_total_rep'] = O16_total
rep_2020['1_percent_rep'] = rep_2020['1_candidate_rep']/rep_2020['1_total_rep']

rep_2020['2_candidate_rep'] = O12_candidate
rep_2020['2_total_rep'] = O12_total
rep_2020['2_percent_rep'] = rep_2020['2_candidate_rep']/rep_2020['2_total_rep']

rep_2020['3_candidate_rep'] = O8_candidate
rep_2020['3_total_rep'] = O8_total
rep_2020['3_percent_rep'] = rep_2020['3_candidate_rep']/rep_2020['3_total_rep']

## Race/Gender/Misc. Training/Testing Sets

In [ ]:
# drop row of county fips that had nans 
##### county fips 51515 belonged to Bedford City, was added to Bedford County
##### of county fips 51019 in 2013, meaning it has missing data for years 2012
##### and 2016 for our dataset

gender_data = gender_data[gender_data['fips'] != 51515]
race_data = race_data[race_data['fips'] != 51515]
urban_data.rename(columns={'FIPS code':'fips'}, inplace=True)
urban_data = urban_data[urban_data['fips'] != 51515]

data_2.drop(columns=['trump16', 'clinton16', 'otherpres16', 'romney12', 'obama12', 'otherpres12', 
                     'demhouse16', 'rephouse16', 'otherhouse16', 'demgov16', 'state', 'county',
                     '%_Repub16', '%_Dem16', '%_Repub12', '%_Dem12'], inplace=True)

In [ ]:
total_2000 = race_data['ai_2000'] + race_data['a_2000'] + race_data['b_2000'] + race_data['w_2000']
total_2004 = race_data['ai_2004'] + race_data['a_2004'] + race_data['b_2004'] + race_data['w_2004']
total_2008 = race_data['ai_2008'] + race_data['a_2008'] + race_data['b_2008'] + race_data['w_2008']
total_2012 = race_data['ai_2012'] + race_data['a_2012'] + race_data['b_2012'] + race_data['w_2012']
total_2016 = race_data['ai_2016'] + race_data['a_2016'] + race_data['b_2016'] + race_data['w_2016']

race_data['ai_2000_%'] = race_data['ai_2000'] / total_2000
race_data['ai_2004_%'] = race_data['ai_2004'] / total_2004
race_data['ai_2008_%'] = race_data['ai_2008'] / total_2008
race_data['ai_2012_%'] = race_data['ai_2012'] / total_2012
race_data['ai_2016_%'] = race_data['ai_2016'] / total_2016

race_data['w_2000_%'] = race_data['w_2000'] / total_2000
race_data['w_2004_%'] = race_data['w_2004'] / total_2004
race_data['w_2008_%'] = race_data['w_2008'] / total_2008
race_data['w_2012_%'] = race_data['w_2012'] / total_2012
race_data['w_2016_%'] = race_data['w_2016'] / total_2016

race_data['b_2000_%'] = race_data['b_2000'] / total_2000
race_data['b_2004_%'] = race_data['b_2004'] / total_2004
race_data['b_2008_%'] = race_data['b_2008'] / total_2008
race_data['b_2012_%'] = race_data['b_2012'] / total_2012
race_data['b_2016_%'] = race_data['b_2016'] / total_2016

race_data['a_2000_%'] = race_data['a_2000'] / total_2000
race_data['a_2004_%'] = race_data['a_2004'] / total_2004
race_data['a_2008_%'] = race_data['a_2008'] / total_2008
race_data['a_2012_%'] = race_data['a_2012'] / total_2012
race_data['a_2016_%'] = race_data['a_2016'] / total_2016


In [ ]:
############################################################################

# gender portion of training set
gender_X = pd.DataFrame(
    data=gender_data[['fips','f_2000','m_2000','f_2004','m_2004','f_2008','m_2008']]
    ).rename(columns={'f_2000':'3_f','m_2000':'3_m','f_2004':'2_f','m_2004':'2_m','f_2008':'1_f','m_2008':'1_m'})

num_attribs = gender_X.columns.drop(['fips'])

full_pipeline = ColumnTransformer([
        ("std_scaler", StandardScaler(), num_attribs),
    ], remainder='drop')

# columns the pipeline produces
fips = gender_X['fips']
gender_X = pd.DataFrame(full_pipeline.fit_transform(gender_X), columns=num_attribs).set_index(gender_X.index)
gender_X['fips'] = fips


############################################################################

# race portion of training set
race_X = pd.DataFrame(
    data=race_data[['fips','ai_2000_%','ai_2004_%','ai_2008_%','a_2000_%','a_2004_%','a_2008_%','b_2000_%','b_2004_%','b_2008_%','w_2000_%','w_2004_%','w_2008_%']]
    ).rename(columns={'ai_2000_%':'3_ai','ai_2004_%':'2_ai','ai_2008_%':'1_ai','a_2000_%':'3_a','a_2004_%':'2_a','a_2008_%':'1_a','b_2000_%':'3_b','b_2004_%':'2_b','b_2008_%':'1_b','w_2000_%':'3_w','w_2004_%':'2_w','w_2008_%':'1_w'})

change_ai = race_data['ai_2008_%'] - race_data['ai_2000_%']
change_a = race_data['a_2008_%'] - race_data['a_2000_%']
change_b = race_data['b_2008_%'] - race_data['b_2000_%']
change_w = race_data['w_2008_%'] - race_data['w_2000_%']


race = race_X.columns.drop(['fips'])

race_pipeline = ColumnTransformer([
        ('std_scaler', StandardScaler(), race),
    ], remainder='drop')

fips = race_X['fips']
race_X = pd.DataFrame(race_pipeline.fit_transform(race_X), columns=race).set_index(race_X.index)
race_X['fips'] = fips

race_X['change_ai'] = change_ai
race_X['change_a'] = change_a
race_X['change_b'] = change_b
race_X['change_w'] = change_w

race_X['fips'] = race_X['fips'].astype(int)


In [ ]:
############################################################################

# gender portion of testing set
gender_X_test = pd.DataFrame(
    data=gender_data[['fips','f_2004','m_2004','f_2008','m_2008','f_2012','m_2012',]]
    ).rename(columns={'f_2004':'3_f','m_2004':'3_m','f_2008':'2_f','m_2008':'2_m','f_2012':'1_f','m_2012':'1_m'})


# columns the pipeline produces
fips = gender_X_test['fips']
gender_X_test = pd.DataFrame(full_pipeline.transform(gender_X_test), columns=num_attribs).set_index(gender_X_test.index)
gender_X_test['fips'] = fips

############################################################################

# race portion of training set
race_X_test = pd.DataFrame(
    data=race_data[['fips','ai_2004_%','ai_2008_%','ai_2012_%','a_2004_%','a_2008_%','a_2012_%','b_2004_%','b_2008_%','b_2012_%','w_2004_%','w_2008_%','w_2012_%']]
    ).rename(columns={'ai_2004_%':'3_ai','ai_2008_%':'2_ai','ai_2012_%':'1_ai',
                      'a_2004_%':'3_a','a_2008_%':'2_a','a_2012_%':'1_a',
                      'b_2004_%':'3_b','b_2008_%':'2_b','b_2012_%':'1_b',
                      'w_2004_%':'3_w','w_2008_%':'2_w','w_2012_%':'1_w'})

change_ai = race_data['ai_2012_%'] - race_data['ai_2004_%']
change_a = race_data['a_2012_%'] - race_data['a_2004_%']
change_b = race_data['b_2012_%'] - race_data['b_2004_%']
change_w = race_data['w_2012_%'] - race_data['w_2004_%']


fips = race_X_test['fips']
race_X_test = pd.DataFrame(race_pipeline.transform(race_X_test), columns=race).set_index(race_X_test.index)
race_X_test['fips'] = fips
race_X_test['change_ai'] = change_ai
race_X_test['change_a'] = change_a
race_X_test['change_b'] = change_b
race_X_test['change_w'] = change_w


In [ ]:
############################################################################

# gender portion of 2020 set
gender_2020 = pd.DataFrame(
    data=gender_data[['fips','f_2008','m_2008','f_2012','m_2012','f_2016','m_2016',]]
    ).rename(columns={'f_2008':'3_f','m_2008':'3_m','f_2012':'2_f','m_2012':'2_m','f_2016':'1_f','m_2016':'1_m'})


# columns the pipeline produces
fips = gender_2020['fips']
gender_2020 = pd.DataFrame(full_pipeline.transform(gender_2020), columns=num_attribs).set_index(gender_2020.index)
gender_2020['fips'] = fips

############################################################################

# race portion of 2020 set
race_2020 = pd.DataFrame(
    data=race_data[['fips','ai_2008_%','ai_2012_%','ai_2016_%','a_2008_%','a_2012_%','a_2016_%','b_2008_%','b_2012_%','b_2016_%','w_2008_%','w_2012_%','w_2016_%']]
    ).rename(columns={'ai_2008_%':'3_ai','ai_2012_%':'2_ai','ai_2016_%':'1_ai',
                      'a_2008_%':'3_a','a_2012_%':'2_a','a_2016_%':'1_a',
                      'b_2008_%':'3_b','b_2012_%':'2_b','b_2016_%':'1_b',
                      'w_2008_%':'3_w','w_2012_%':'2_w','w_2016_%':'1_w'})

change_ai = race_data['ai_2016_%'] - race_data['ai_2008_%']
change_a = race_data['a_2016_%'] - race_data['a_2008_%']
change_b = race_data['b_2016_%'] - race_data['b_2008_%']
change_w = race_data['w_2016_%'] - race_data['w_2008_%']


fips = race_2020['fips']
race_2020 = pd.DataFrame(race_pipeline.transform(race_2020), columns=race).set_index(race_2020.index)
race_2020['fips'] = fips
race_2020['change_ai'] = change_ai
race_2020['change_a'] = change_a
race_2020['change_b'] = change_b
race_2020['change_w'] = change_w


## Combining Training/Testing Sets

In [ ]:
X_train = democrat_X.copy()
X_train = pd.merge(X_train, republican_X, on='fips')
X_train = pd.merge(X_train, gender_X, on='fips')
X_train = pd.merge(X_train, race_X, on='fips')
X_train = pd.merge(X_train, urban_data, on='fips')
X_train = pd.merge(X_train, data_2, on='fips')

X_test = democrat_X_test.copy()
X_test = pd.merge(X_test, republican_X_test, on='fips')
X_test = pd.merge(X_test, gender_X_test, on='fips')
X_test = pd.merge(X_test, race_X_test, on='fips')
X_test = pd.merge(X_test, urban_data, on='fips')
X_test = pd.merge(X_test, data_2, on='fips')

data_2020 = dem_2020.copy()
data_2020 = pd.merge(data_2020, rep_2020, on='fips')
data_2020 = pd.merge(data_2020, gender_2020, on='fips')
data_2020 = pd.merge(data_2020, race_2020, on='fips')
data_2020 = pd.merge(data_2020, urban_data, on='fips')
data_2020 = pd.merge(data_2020, data_2, on='fips')

#Reorganizing Test and Train data after merge
List of columns described below


1.   fips: fips county code
2.   1_total: total number of voters in first election
3.   2_total: total number of voters in second election
4.   3_total: total number of voters in third election
5.   1_candidate_dem: number of democrat votes in the first election
6.   1_precent_dem: percentage of democrat votes in first election
7.   2_candidate_dem: number of democrat votes in the second election
8.   2_precent_dem: percentage of democrat votes in second election
9.   3_candidate_dem: number of democrat votes in the third election
10.  3_precent_dem: percentage of democrat votes in third election
11.  1_candidate_rep: number of republican votes in the first election
12.  1_precent_rep: percentage of republican votes in first election
13.  2_candidate_rep: number of republican votes in the second election
14.  2_precent_rep: percentage of republican votes in second election
15.  3_candidate_rep: number of republican votes in the third election
16.  3_precent_rep: percentage of republican votes in third election
17.  3_f: standardized female votes in third election
18.  3_m: standardized male votes in third election
19.  2_f: standardized female votes in second election
20.  2_m: standardized male votes in second election
21.  1_f: standardized female votes in first election
22.  1_m: male votes in first election
23.  3_ai: standardized American Indian votes in third election
24.  2_ai: standardized American Indian votes in second election 
25.  1_ai: standardized American Indian votes in first election
26.  3_a: standardized Asian votes in third election
27.  2_a: standardized Asian votes in second election
28.  1_a: standardized Asian votes in first election
29.  3_b: standardized Black votes in third election
30.  2_b: standardized Black votes in second election
31.  1_b: standardized Black votes in first election
32.  3_w: standardized White votes in third election
33.  2_w: standardized White votes in second election
34.  1_w: standardized White votes in first election


In [ ]:
train_copy=X_train.drop(columns=['1_total_rep', '2_total_rep', '3_total_rep'])
train_copy=train_copy.rename(columns={'1_total_dem':'1_total','2_total_dem':'2_total','3_total_dem':'3_total'})
train_copy= train_copy[["fips", "1_total", "2_total","3_total","1_candidate_dem","1_percent_dem","2_candidate_dem","2_percent_dem","3_candidate_dem","3_percent_dem",
            "1_candidate_rep","1_percent_rep","2_candidate_rep","2_percent_rep","3_candidate_rep","3_percent_rep","3_f","3_m",
            "2_f","2_m","1_f","1_m", "3_ai","2_ai","1_ai","3_a","2_a","1_a","3_b","2_b","1_b","3_w","2_w","1_w","change_ai","change_a","change_b","change_w",
            "2013_code", "2006_code", "1990_code","total_population","cvap","white_pct","black_pct","hispanic_pct","nonwhite_pct","foreignborn_pct",
            "female_pct","age29andunder_pct","age65andolder_pct","median_hh_inc","clf_unemploy_pct","lesshs_pct","lesscollege_pct","lesshs_whites_pct",
            "lesscollege_whites_pct","rural_pct","ruralurban_cc"]]

test_copy=X_test.drop(columns=['1_total_rep', '2_total_rep', '3_total_rep'])
test_copy=test_copy.rename(columns={'1_total_dem':'1_total','2_total_dem':'2_total','3_total_dem':'3_total'})
test_copy= test_copy[["fips", "1_total", "2_total","3_total","1_candidate_dem","1_percent_dem","2_candidate_dem","2_percent_dem","3_candidate_dem","3_percent_dem",
            "1_candidate_rep","1_percent_rep","2_candidate_rep","2_percent_rep","3_candidate_rep","3_percent_rep","3_f","3_m",
            "2_f","2_m","1_f","1_m", "3_ai","2_ai","1_ai","3_a","2_a","1_a","3_b","2_b","1_b","3_w","2_w","1_w","change_ai","change_a","change_b","change_w",
            "2013_code", "2006_code", "1990_code","total_population","cvap","white_pct","black_pct","hispanic_pct","nonwhite_pct","foreignborn_pct",
            "female_pct","age29andunder_pct","age65andolder_pct","median_hh_inc","clf_unemploy_pct","lesshs_pct","lesscollege_pct","lesshs_whites_pct",
            "lesscollege_whites_pct","rural_pct","ruralurban_cc"]]

data2020_copy=data_2020.drop(columns=['1_total_rep', '2_total_rep', '3_total_rep'])
data2020_copy=data2020_copy.rename(columns={'1_total_dem':'1_total','2_total_dem':'2_total','3_total_dem':'3_total'})
data2020_copy= data2020_copy[["fips", "1_total", "2_total","3_total","1_candidate_dem","1_percent_dem","2_candidate_dem","2_percent_dem","3_candidate_dem","3_percent_dem",
            "1_candidate_rep","1_percent_rep","2_candidate_rep","2_percent_rep","3_candidate_rep","3_percent_rep","3_f","3_m",
            "2_f","2_m","1_f","1_m", "3_ai","2_ai","1_ai","3_a","2_a","1_a","3_b","2_b","1_b","3_w","2_w","1_w","change_ai","change_a","change_b","change_w",
            "2013_code", "2006_code", "1990_code","total_population","cvap","white_pct","black_pct","hispanic_pct","nonwhite_pct","foreignborn_pct",
            "female_pct","age29andunder_pct","age65andolder_pct","median_hh_inc","clf_unemploy_pct","lesshs_pct","lesscollege_pct","lesshs_whites_pct",
            "lesscollege_whites_pct","rural_pct","ruralurban_cc"]]            

In [ ]:
train_copy

,fips,1_total,2_total,3_total,1_candidate_dem,1_percent_dem,2_candidate_dem,2_percent_dem,3_candidate_dem,3_percent_dem,1_candidate_rep,1_percent_rep,2_candidate_rep,2_percent_rep,3_candidate_rep,3_percent_rep,3_f,3_m,2_f,2_m,1_f,1_m,3_ai,2_ai,1_ai,3_a,2_a,1_a,3_b,2_b,1_b,3_w,2_w,1_w,change_ai,change_a,change_b,change_w,2013_code,2006_code,1990_code,total_population,cvap,white_pct,black_pct,hispanic_pct,nonwhite_pct,foreignborn_pct,female_pct,age29andunder_pct,age65andolder_pct,median_hh_inc,clf_unemploy_pct,lesshs_pct,lesscollege_pct,lesshs_whites_pct,lesscollege_whites_pct,rural_pct,ruralurban_cc
0,51001,25814,22997,20327,8091.0,0.313435,6862.0,0.298387,6659.0,0.327594,17444.0,0.675757,15891.0,0.691003,13162.0,0.647513,-0.144907,-0.150124,-0.181347,-0.186127,-0.218889,-0.223166,0.008709,0.048909,0.168392,-0.474858,-0.469522,-0.430418,0.702728,0.629368,0.559789,-0.639861,-0.558100,-0.484563,0.002214,0.003199,-0.028287,0.022874,6,6,6,33060.0,24490.0,60.589837,28.877798,8.802178,39.410163,6.557774,51.225045,33.360557,20.955838,38503.0,7.030120,19.496855,81.664570,13.222930,75.554140,100.000000,8
1,51003,12919,9954,8351,7163.0,0.554455,4878.0,0.490054,4314.0,0.516585,5617.0,0.434786,4999.0,0.502210,3873.0,0.463777,0.314809,0.271246,0.329379,0.286795,0.357304,0.321193,-0.230279,-0.211431,-0.170283,0.941364,0.950909,0.963340,-0.580823,-0.543094,-0.544638,0.467548,0.405319,0.374121,0.001695,0.014567,0.005259,-0.021522,4,4,4,104287.0,77380.0,77.424799,9.319474,5.592260,22.575201,9.917823,52.107166,39.463212,16.454592,70342.0,3.244575,8.612515,48.466284,6.756994,44.724448,45.022734,3
2,51005,12579,12128,11001,4109.0,0.326656,3888.0,0.320580,3638.0,0.330697,8187.0,0.650847,8173.0,0.673895,7142.0,0.649214,-0.352568,-0.350923,-0.366606,-0.364717,-0.377610,-0.376387,-0.241287,-0.348066,-0.445264,-0.530875,-0.557693,-0.551545,-0.822294,-0.833476,-0.837624,0.897813,0.930586,0.946677,-0.000033,0.000718,-0.002943,0.002258,6,6,6,15919.0,12705.0,91.940449,5.207614,1.432251,8.059551,1.300333,51.077329,31.660280,23.902255,45538.0,4.560986,15.537543,83.711604,15.080608,83.242745,52.393846,6
3,51007,4153,3439,3066,2838.0,0.683361,2155.0,0.626636,1981.0,0.646119,1288.0,0.310137,1254.0,0.364641,1023.0,0.333659,-0.411408,-0.404202,-0.413476,-0.409106,-0.412236,-0.409973,-0.089646,0.029871,0.121673,-0.543708,-0.556842,-0.550926,0.482979,0.375520,0.263655,-0.407915,-0.290832,-0.167456,0.002609,0.001023,-0.040777,0.037145,2,2,6,12793.0,10060.0,72.203549,23.528492,1.188150,27.796451,0.242320,48.080982,34.565778,18.166185,58269.0,4.030710,18.747199,85.925594,20.341615,85.931677,100.000000,1
4,51009,162088,133814,110951,74310.0,0.458455,49346.0,0.368766,38638.0,0.348244,86413.0,0.533124,83745.0,0.625831,69924.0,0.630224,-0.202689,-0.216172,-0.219604,-0.231776,-0.228332,-0.239770,0.694115,0.713357,0.573251,-0.456558,-0.470934,-0.480692,0.021665,0.022391,0.014845,0.008456,0.020203,0.049473,-0.000017,0.001245,-0.003491,0.002263,3,4,4,31999.0,25285.0,75.611738,17.844308,2.184443,24.388262,1.712554,52.042251,34.785462,18.866215,47002.0,5.526959,17.252723,81.120249,14.843572,79.955469,63.663339,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,51800,35830,31404,26149,11017.0,0.307480,9102.0,0.289836,8160.0,0.312058,24420.0,0.681552,21925.0,0.698159,17224.0,0.658687,0.122286,0.088135,0.194833,0.166801,0.225917,0.195494,-0.035400,-0.115328,-0.144946,-0.210899,-0.087627,-0.023793,1.409970,1.329793,1.392411,-1.376945,-1.308155,-1.371292,0.000471,0.009836,-0.008605,-0.001701,2,2,2,87061.0,64145.0,50.043073,41.368696,3.750244,49.956927,3.720380,51.592562,39.613604,12.974811,65435.0,7.276632,11.963259,73.409417,7.570411,68.804132,21.632677,1
129,51810,2959,2865,2688,864.0,0.291991,846.0,0.295288,851.0,0.316592,2031.0,0.686381,1962.0,0.684817,1759.0,0.654390,3.622110,3.644335,3

In [ ]:
test_copy

,fips,1_total,2_total,3_total,1_candidate_dem,1_percent_dem,2_candidate_dem,2_percent_dem,3_candidate_dem,3_percent_dem,1_candidate_rep,1_percent_rep,2_candidate_rep,2_percent_rep,3_candidate_rep,3_percent_rep,3_f,3_m,2_f,2_m,1_f,1_m,3_ai,2_ai,1_ai,3_a,2_a,1_a,3_b,2_b,1_b,3_w,2_w,1_w,change_ai,change_a,change_b,change_w,2013_code,2006_code,1990_code,total_population,cvap,white_pct,black_pct,hispanic_pct,nonwhite_pct,foreignborn_pct,female_pct,age29andunder_pct,age65andolder_pct,median_hh_inc,clf_unemploy_pct,lesshs_pct,lesscollege_pct,lesshs_whites_pct,lesscollege_whites_pct,rural_pct,ruralurban_cc
0,51001,25696,25814,22997,7595.0,0.295571,8091.0,0.313435,6862.0,0.298387,17695.0,0.688629,17444.0,0.675757,15891.0,0.691003,-0.163411,-0.168677,-0.203999,-0.206750,-0.226916,-0.227624,0.135407,0.234194,0.260345,-0.423397,-0.391349,-0.402379,0.616721,0.551176,0.563624,-0.565956,-0.499643,-0.496909,0.001914,0.002916,-0.012781,0.007951,6,6,6,33060.0,24490.0,60.589837,28.877798,8.802178,39.410163,6.557774,51.225045,33.360557,20.955838,38503.0,7.030120,19.496855,81.664570,13.222930,75.554140,100.000000,8
1,51003,13652,12919,9954,7276.0,0.532962,7163.0,0.554455,4878.0,0.490054,6151.0,0.450557,5617.0,0.434786,4999.0,0.502210,0.376091,0.325804,0.395952,0.360820,0.401199,0.354977,-0.122547,-0.080466,-0.126269,1.253518,1.241011,1.057495,-0.549803,-0.543661,-0.547347,0.392093,0.356271,0.358254,0.001215,0.010518,-0.000559,-0.011174,4,4,4,104287.0,77380.0,77.424799,9.319474,5.592260,22.575201,9.917823,52.107166,39.463212,16.454592,70342.0,3.244575,8.612515,48.466284,6.756994,44.724448,45.022734,3
2,51005,12918,12579,12128,3685.0,0.285261,4109.0,0.326656,3888.0,0.320580,8736.0,0.676266,8187.0,0.650847,8173.0,0.673895,-0.359108,-0.355409,-0.369264,-0.366504,-0.381090,-0.379527,-0.257931,-0.335950,-0.420012,-0.527489,-0.533212,-0.527031,-0.838715,-0.834103,-0.846252,0.914432,0.926980,0.949877,0.000252,0.001397,-0.001577,-0.000072,6,6,6,15919.0,12705.0,91.940449,5.207614,1.432251,8.059551,1.300333,51.077329,31.660280,23.902255,45538.0,4.560986,15.537543,83.711604,15.080608,83.242745,52.393846,6
3,51007,4232,4153,3439,2772.0,0.655009,2838.0,0.683361,2155.0,0.626636,1396.0,0.329868,1288.0,0.310137,1254.0,0.364641,-0.408619,-0.401821,-0.405318,-0.401522,-0.410085,-0.408872,0.116543,0.190788,0.181255,-0.526485,-0.532487,-0.463067,0.364159,0.257614,0.195673,-0.300178,-0.183558,-0.118047,0.001529,0.003343,-0.031837,0.026965,2,2,6,12793.0,10060.0,72.203549,23.528492,1.188150,27.796451,0.242320,48.080982,34.565778,18.166185,58269.0,4.030710,18.747199,85.925594,20.341615,85.931677,100.000000,1
4,51009,170988,162088,133814,77694.0,0.454383,74310.0,0.458455,49346.0,0.368766,90934.0,0.531815,86413.0,0.533124,83745.0,0.625831,-0.203824,-0.216408,-0.213832,-0.224061,-0.226636,-0.239391,0.793766,0.610346,0.675897,-0.425064,-0.450229,-0.419533,0.012819,0.010964,0.011858,0.009124,0.032670,0.037550,-0.000054,0.002426,-0.002472,0.000100,3,4,4,31999.0,25285.0,75.611738,17.844308,2.184443,24.388262,1.712554,52.042251,34.785462,18.866215,47002.0,5.526959,17.252723,81.120249,14.843572,79.955469,63.663339,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,51800,37425,35830,31404,10209.0,0.272786,11017.0,0.307480,9102.0,0.289836,26679.0,0.712866,24420.0,0.681552,21925.0,0.698159,0.233964,0.200339,0.259147,0.229762,0.249351,0.219226,-0.027326,-0.056926,-0.096618,0.027456,0.084888,0.030385,1.313600,1.376570,1.425882,-1.311830,-1.383511,-1.416047,0.000730,0.006199,0.013738,-0.020667,2,2,2,87061.0,64145.0,50.043073,41.368696,3.750244,49.956927,3.720380,51.592562,39.613604,12.974811,65435.0,7.276632,11.963259,73.409417,7.570411,68.804132,21.632677,1
129,51810,2948,2959,2865,735.0,0.249322,864.0,0.291991,846.0,0.295288,2144.0,0.727273,2031.0,0.686381,1962.0,0.684817,3.786839,3.683921,3.5

# Models

XGBoost

In [ ]:
from xgboost import XGBRegressor
import math 
from sklearn.metrics import mean_squared_error

m = XGBRegressor(
    reg='squarederror'
)

####################################################
m.fit(X_train, y_train_democrat['percent'])

print("\nDemocrat: \n")


y_pred = m.predict(X_train)
print("Train RMSE: {}".format(math.sqrt(mean_squared_error(y_train_democrat['percent'], y_pred))))

y_pred = m.predict(X_test)
print("Test RMSE: {}\n".format(math.sqrt(mean_squared_error(y_test_democrat['percent'], y_pred))))

#####################################################
m.fit(X_train, y_train_republican['percent'])

print("\nRepublican: \n")


y_pred = m.predict(X_train)
print("Train RMSE: {}".format(math.sqrt(mean_squared_error(y_train_republican['percent'], y_pred))))

y_pred = m.predict(X_test)
print("Test RMSE: {}".format(math.sqrt(mean_squared_error(y_test_republican['percent'], y_pred))))

print()
print(m)

[21:46:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.

Democrat: 

Train RMSE: 0.003778116164107986
Test RMSE: 0.13617762572616124

[21:46:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.

Republican: 

Train RMSE: 0.003493856845419896
Test RMSE: 0.13422428347424167

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg='squarederror', reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)


Tuning XGBboost

In [ ]:
param = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2),
 'n_estimators': range(100,150,5),
 'gamma': [0,.1,.2],
 'subsample': [.8,.9,1],
 'colsample_bytree ':[.5,.6,.7,.8,.9,1]
}
gsearch = GridSearchCV(estimator = XGBRegressor(reg='squarederror'), 
 param_grid = param, scoring='neg_mean_squared_error',n_jobs=-1,verbose = 0, cv=2)

gsearch.fit(train_copy,y_train_democrat['percent'])

model_xgb = gsearch.best_estimator_
print("\nDemocrat: \n")

y_pred = model_xgb.predict(train_copy)
print("Train RMSE: {}".format(math.sqrt(mean_squared_error(y_train_democrat['percent'], y_pred))))

y_pred = model_xgb.predict(test_copy)
print("Test RMSE: {}".format(math.sqrt(mean_squared_error(y_test_democrat['percent'], y_pred))))

print()
print(model_xgb)

#####################################################
model_xgb.fit(train_copy, y_train_republican['percent'])

print("\nRepublican: \n")


y_pred = model_xgb.predict(train_copy)
print("Train RMSE: {}".format(math.sqrt(mean_squared_error(y_train_republican['percent'], y_pred))))

y_pred = model_xgb.predict(test_copy)
print("Test RMSE: {}".format(math.sqrt(mean_squared_error(y_test_republican['percent'], y_pred))))

print()
print(model_xgb)


[21:59:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.

Democrat: 

Train RMSE: 0.0008787870180946765
Test RMSE: 0.13603172420268114

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, colsample_bytree =0.5,
             gamma=0, importance_type='gain', learning_rate=0.1,
             max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
             n_estimators=100, n_jobs=1, nthread=None, objective='reg:linear',
             random_state=0, reg='squarederror', reg_alpha=0, reg_lambda=1,
             scale_pos_weight=1, seed=None, silent=None, subsample=0.9,
             verbosity=1)
[21:59:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.

Republican: 

Train RMSE: 0.0008676350866648303
Test RMSE: 0.13485404545891833

XGBRegressor(base_score=0.5, booster='gbtree',

Random Forest

In [ ]:
from sklearn.metrics import mean_squared_error
import math

m = RandomForestRegressor(random_state=42, n_jobs=-1)
 

####################################################
m.fit(X_train, y_train_democrat['percent'])

print("\nDemocrat: \n")


y_pred = m.predict(X_train)
print("Train RMSE: {}".format(math.sqrt(mean_squared_error(y_train_democrat['percent'], y_pred))))

y_pred = m.predict(X_test)
print("Test RMSE: {}\n".format(math.sqrt(mean_squared_error(y_test_democrat['percent'], y_pred))))

#####################################################
m.fit(X_train, y_train_republican['percent'])

print("\nRepublican: \n")


y_pred = m.predict(X_train)
print("Train RMSE: {}".format(math.sqrt(mean_squared_error(y_train_republican['percent'], y_pred))))

y_pred = m.predict(X_test)
print("Test RMSE: {}".format(math.sqrt(mean_squared_error(y_test_republican['percent'], y_pred))))

print()
print(m)


Democrat: 

Train RMSE: 0.012155078162482546
Test RMSE: 0.13506037226470638


Republican: 

Train RMSE: 0.011637807986555504
Test RMSE: 0.13339834667162673

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)


Tuning Random Forest

In [ ]:
param_grid = {
    'max_depth': [None, 80, 100],
    'max_features': [5, 10, 'auto'],
    'warm_start' : [False, True],
    'min_samples_leaf': [1, 4, 7],
    'n_estimators': [10, 50, 100, 200],
    'random_state': [42]
}

rf = RandomForestRegressor(random_state=42)

# complete grid search
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 2, n_jobs = -1, verbose = 0, 
                           scoring='neg_mean_squared_error')
# fit and get best
grid_search.fit(X_train,y_train_democrat['percent'])
model_rf = grid_search.best_estimator_

print("Democrat")
y_pred = model_rf.predict(X_train)
print("Train RMSE: {}".format(math.sqrt(mean_squared_error(y_train_democrat['percent'], y_pred))))

y_pred = model_rf.predict(X_test)
print("Test RMSE: {}".format(math.sqrt(mean_squared_error(y_test_democrat['percent'], y_pred))))

print()
print(model_rf)

###################### repub
grid_search.fit(X_train,y_train_republican['percent'])
model_rf = grid_search.best_estimator_

print("Republican")
y_pred = model_rf.predict(X_train)
print("Train RMSE: {}".format(math.sqrt(mean_squared_error(y_train_republican['percent'], y_pred))))

y_pred = model_rf.predict(X_test)
print("Test RMSE: {}".format(math.sqrt(mean_squared_error(y_test_republican['percent'], y_pred))))

print()
print(model_rf)

Democrat
Train RMSE: 0.012699697937222134
Test RMSE: 0.1349079418633087

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=50, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)
Republican
Train RMSE: 0.011637807986555506
Test RMSE: 0.13339834667162673

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,


Linear Regression

In [ ]:


m = LinearRegression(n_jobs=-1)
 

####################################################
m.fit(X_train, y_train_democrat['percent'])

print("\nDemocrat: \n")


y_pred = m.predict(X_train)
print("Train RMSE: {}".format(math.sqrt(mean_squared_error(y_train_democrat['percent'], y_pred))))

y_pred = m.predict(X_test)
print("Test RMSE: {}\n".format(math.sqrt(mean_squared_error(y_test_democrat['percent'], y_pred))))

#####################################################
m.fit(X_train, y_train_republican['percent'])

print("\nRepublican: \n")


y_pred = m.predict(X_train)
print("Train RMSE: {}".format(math.sqrt(mean_squared_error(y_train_republican['percent'], y_pred))))

y_pred = m.predict(X_test)
print("Test RMSE: {}".format(math.sqrt(mean_squared_error(y_test_republican['percent'], y_pred))))

print()
print(m)


Democrat: 

Train RMSE: 0.015694097280404436
Test RMSE: 0.1352073462273473


Republican: 

Train RMSE: 0.015955610129138754
Test RMSE: 0.13963398625026763

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False)


Elastic-Net


In [ ]:
from sklearn.linear_model import ElasticNet
elastic_net = ElasticNet(tol=100,random_state=42)
elastic_net.fit(train_copy, y_train_democrat['percent'])
print("\nDemocrat: \n")
y_pred = elastic_net.predict(train_copy)
print("Train RMSE: {}".format(math.sqrt(mean_squared_error(y_train_democrat['percent'], y_pred))))

y_pred = elastic_net.predict(test_copy)
print("Test RMSE: {}\n".format(math.sqrt(mean_squared_error(y_test_democrat['percent'], y_pred))))

#####################################################
elastic_net.fit(train_copy, y_train_republican['percent'])

print("\nRepublican: \n")


y_pred = elastic_net.predict(train_copy)
print("Train RMSE: {}".format(math.sqrt(mean_squared_error(y_train_republican['percent'], y_pred))))

y_pred = elastic_net.predict(test_copy)
print("Test RMSE: {}".format(math.sqrt(mean_squared_error(y_test_republican['percent'], y_pred))))

print()
print(elastic_net)


Democrat: 

Train RMSE: 0.12768360566676368
Test RMSE: 0.16384309987477408


Republican: 

Train RMSE: 0.12602061140394336
Test RMSE: 0.1627443574701879

ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=42, selection='cyclic', tol=100, warm_start=False)


Tuning Elastic Net 

In [ ]:
param_grid = {
    'tol': range(10,1000,10),
    'alpha': [.1, .5, 1],
    'l1_ratio' : [0,.2,.5,.7,1],
    'warm_start': [False,True],
    'selection': ['cyclic','random'],
    'random_state': [42]
}

elastic_net = ElasticNet(random_state=42)


# complete grid search
grid_search = GridSearchCV(estimator = elastic_net, param_grid = param_grid, 
                          cv = 2, n_jobs = -1, verbose = 0, 
                           scoring='neg_mean_squared_error')
# fit and get best
grid_search.fit(X_train,y_train_democrat['percent'])
model_en = grid_search.best_estimator_
print("\nDemocrat: \n")

y_pred = model_en.predict(X_train)
print("Train RMSE: {}".format(math.sqrt(mean_squared_error(y_train_democrat['percent'], y_pred))))

y_pred = model_en.predict(X_test)
print("Test RMSE: {}".format(math.sqrt(mean_squared_error(y_test_democrat['percent'], y_pred))))

print()
print(model_en)

####################
grid_search.fit(X_train,y_train_republican['percent'])
model_en = grid_search.best_estimator_
print("\nRepublican: \n")

y_pred = model_en.predict(X_train)
print("Train RMSE: {}".format(math.sqrt(mean_squared_error(y_train_republican['percent'], y_pred))))

y_pred = model_en.predict(X_test)
print("Test RMSE: {}".format(math.sqrt(mean_squared_error(y_test_republican['percent'], y_pred))))

print()
print(model_en)





Democrat: 

Train RMSE: 0.098414352567179
Test RMSE: 0.15028587978902713

ElasticNet(alpha=0.1, copy_X=True, fit_intercept=True, l1_ratio=0,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=42, selection='random', tol=10, warm_start=False)

Republican: 

Train RMSE: 0.09700899704703507
Test RMSE: 0.1464583881243164

ElasticNet(alpha=0.1, copy_X=True, fit_intercept=True, l1_ratio=0,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=42, selection='random', tol=10, warm_start=False)


SGDRegressor

In [ ]:
from sklearn import linear_model
sgd = linear_model.SGDRegressor(eta0=0.000000000001)
sgd.fit(X_train, y_train_democrat['percent'])

print("\nDemocrat: \n")
y_pred = sgd.predict(X_train)
print("Train RMSE: {}".format(math.sqrt(mean_squared_error(y_train_democrat['percent'], y_pred))))

y_pred = sgd.predict(X_test)
print("Test RMSE: {}\n".format(math.sqrt(mean_squared_error(y_test_democrat['percent'], y_pred))))

#####################################################
sgd.fit(X_train, y_train_republican['percent'])

print("\nRepublican: \n")


y_pred = sgd.predict(X_train)
print("Train RMSE: {}".format(math.sqrt(mean_squared_error(y_train_republican['percent'], y_pred))))

y_pred = sgd.predict(X_test)
print("Test RMSE: {}".format(math.sqrt(mean_squared_error(y_test_republican['percent'], y_pred))))

print()
print(sgd)



Democrat: 

Train RMSE: 0.16286733814850135
Test RMSE: 0.17961477013782368


Republican: 

Train RMSE: 0.18120686063550917
Test RMSE: 0.20784613893371195

SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
             eta0=1e-12, fit_intercept=True, l1_ratio=0.15,
             learning_rate='invscaling', loss='squared_loss', max_iter=1000,
             n_iter_no_change=5, penalty='l2', power_t=0.25, random_state=None,
             shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0,
             warm_start=False)


SVR

In [ ]:
from sklearn.svm import SVR

m = SVR(C=1.0, epsilon=0.2)
 

####################################################
m.fit(X_train, y_train_democrat['percent'])

print("\nDemocrat: \n")


y_pred = m.predict(X_train)
print("Train RMSE: {}".format(math.sqrt(mean_squared_error(y_train_democrat['percent'], y_pred))))

y_pred = m.predict(X_test)
print("Test RMSE: {}\n".format(math.sqrt(mean_squared_error(y_test_democrat['percent'], y_pred))))

#####################################################
m.fit(X_train, y_train_republican['percent'])

print("\nRepublican: \n")


y_pred = m.predict(X_train)
print("Train RMSE: {}".format(math.sqrt(mean_squared_error(y_train_republican['percent'], y_pred))))

y_pred = m.predict(X_test)
print("Test RMSE: {}".format(math.sqrt(mean_squared_error(y_test_republican['percent'], y_pred))))

print()
print(m)


Democrat: 

Train RMSE: 0.12141265564841128
Test RMSE: 0.166933128307819


Republican: 

Train RMSE: 0.1191021166685177
Test RMSE: 0.1606961519508773

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.2, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)


Deep Learning

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

def baseline_model():
	# create model
  model = Sequential()
  model.add(Dense(70, input_dim=62, kernel_initializer='normal', activation='relu'))
  model.add(Dense(80, input_dim=70, kernel_initializer='normal', activation='relu'))
  model.add(Dense(100, input_dim=80, kernel_initializer='normal', activation='relu'))
  model.add(Dense(140, input_dim=100, kernel_initializer='normal', activation='relu'))
  model.add(Dense(95, input_dim=140, kernel_initializer='normal', activation='relu'))
  model.add(Dense(60, input_dim=95, kernel_initializer='normal', activation='relu'))
  model.add(Dense(30, input_dim=60, kernel_initializer='normal', activation='relu'))
  model.add(Dense(15, input_dim=30, kernel_initializer='normal', activation='relu'))

  model.add(Dense(1, kernel_initializer='normal'))
  # model.add(Dense(1, activation='sigmoid'))

	# Compile model
  model.compile(loss='mean_squared_error', optimizer='sgd')
  return model

# evaluate model
model = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5, verbose=0)


model.fit(X_train, y_train_democrat['percent'])

# evaluate the keras model

print("Democrat: \n")
mse = model.score(X_train, y_train_democrat['percent'])
print("Train RMSE: {}".format(math.sqrt(abs(mse))))

mse = model.score(X_test, y_test_democrat['percent'])
print("Test RMSE: {}".format(math.sqrt(abs(mse))))

#####################################################################
print("\nRepublican:\n")
model = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5, verbose=0)

model.fit(X_train, y_train_republican['percent'])

# evaluate the keras model
mse = model.score(X_train, y_train_republican['percent'])
print("Train RMSE: {}".format(math.sqrt(abs(mse))))

mse = model.score(X_test, y_test_republican['percent'])
print("Test RMSE: {}".format(math.sqrt(abs(mse))))


Using TensorFlow backend.


Democrat: 

Train RMSE: 0.08789926411181989
Test RMSE: 0.15324142530090515

Republican:

Train RMSE: 0.07218602342626043
Test RMSE: 0.14499821486863584


Decision Tree Regressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor

inner_r = DecisionTreeRegressor()
inner_r.fit(X_train, y_train_republican['percent'])
y_pred_r = inner_r.predict(X_train)

inner_d = DecisionTreeRegressor()
inner_d.fit(X_train, y_train_democrat['percent'])
y_pred_d = 1-inner_d.predict(X_train)

inter = pd.DataFrame(data=[y_pred_d, y_pred_r]).T

outer = DecisionTreeRegressor(criterion='mse')
outer.fit(inter, y_train_republican['percent'])

y_pred = outer.predict(inter)
print("Train RMSE: {}".format(math.sqrt(mean_squared_error(y_train_republican['percent'], y_pred))))

y_pred = outer.predict(pd.DataFrame(data=[1-inner_d.predict(X_test), inner_r.predict(X_test)]).T)
print("Test RMSE: {}".format(math.sqrt(mean_squared_error(y_test_republican['percent'], y_pred))))



Train RMSE: 0.00010077982548875297
Test RMSE: 0.14156722104314162


Elastic Net CV

In [ ]:
from sklearn.linear_model import ElasticNetCV
from sklearn.datasets import make_regression

e_modelcv = ElasticNetCV(random_state=0, cv=5, max_iter = 100000, tol=100)

#Democrat
e_modelcv.fit(train_copy, y_train_democrat['percent'])

print("\nDemocrat: \n")
y_pred = e_modelcv.predict(train_copy)
print("Train RMSE: {}".format(math.sqrt(mean_squared_error(y_train_democrat['percent'], y_pred))))

y_pred = e_modelcv.predict(test_copy)
print("Test RMSE: {}\n".format(math.sqrt(mean_squared_error(y_test_democrat['percent'], y_pred))))

#####################################################
e_modelcv.fit(train_copy, y_train_republican['percent'])

print("\nRepublican: \n")


y_pred = e_modelcv.predict(train_copy)
print("Train RMSE: {}".format(math.sqrt(mean_squared_error(y_train_republican['percent'], y_pred))))

y_pred = e_modelcv.predict(test_copy)
print("Test RMSE: {}".format(math.sqrt(mean_squared_error(y_test_republican['percent'], y_pred))))

print()
print(e_modelcv)


Democrat: 

Train RMSE: 0.12770638073615592
Test RMSE: 0.16388850780031378


Republican: 

Train RMSE: 0.12603614611546168
Test RMSE: 0.16278418018327975

ElasticNetCV(alphas=None, copy_X=True, cv=5, eps=0.001, fit_intercept=True,
             l1_ratio=0.5, max_iter=100000, n_alphas=100, n_jobs=None,
             normalize=False, positive=False, precompute='auto', random_state=0,
             selection='cyclic', tol=100, verbose=0)


Tuning Elastic Net CV

In [ ]:
param_grid1 = {
    'tol': [0.01, 1, 10, 100],
    'l1_ratio': [0.01, .1,.5,.7,.9, 0.99],
    'max_iter': [100, 1000],
    'random_state': [0, 42]
}

e_modelcv = ElasticNetCV(alphas = [0.1, 0.25, 0.5, 0.75, 1, 2, 4])

# complete grid search
grid_search = GridSearchCV(estimator = e_modelcv, param_grid = param_grid1, cv = 5, n_jobs = -1, verbose = 0, scoring='neg_mean_squared_error')
# fit and get best

grid_search.fit(train_copy,y_train_democrat['percent'])
model_elastic_cv = grid_search.best_estimator_
print("\nDemocrat: \n")

y_pred = model_elastic_cv.predict(train_copy)
print("Train RMSE: {}".format(math.sqrt(mean_squared_error(y_train_democrat['percent'], y_pred))))

y_pred = model_elastic_cv.predict(test_copy)
print("Test RMSE: {}".format(math.sqrt(mean_squared_error(y_test_democrat['percent'], y_pred))))

print()
print(model_elastic_cv)

####################
grid_search.fit(train_copy, y_train_republican['percent'])
model_elastic_cv = grid_search.best_estimator_
print("\nRepublican: \n")

y_pred = model_elastic_cv.predict(train_copy)
print("Train RMSE: {}".format(math.sqrt(mean_squared_error(y_train_republican['percent'], y_pred))))

y_pred = model_elastic_cv.predict(test_copy)
print("Test RMSE: {}".format(math.sqrt(mean_squared_error(y_test_republican['percent'], y_pred))))

print()
print(model_elastic_cv)


Democrat: 

Train RMSE: 0.08056496843521856
Test RMSE: 0.14286470277351723

ElasticNetCV(alphas=[0.1, 0.25, 0.5, 0.75, 1, 2, 4], copy_X=True, cv=None,
             eps=0.001, fit_intercept=True, l1_ratio=0.01, max_iter=100,
             n_alphas=100, n_jobs=None, normalize=False, positive=False,
             precompute='auto', random_state=0, selection='cyclic', tol=10,
             verbose=0)

Republican: 

Train RMSE: 0.07948840912826745
Test RMSE: 0.13964207852591834

ElasticNetCV(alphas=[0.1, 0.25, 0.5, 0.75, 1, 2, 4], copy_X=True, cv=None,
             eps=0.001, fit_intercept=True, l1_ratio=0.01, max_iter=100,
             n_alphas=100, n_jobs=None, normalize=False, positive=False,
             precompute='auto', random_state=0, selection='cyclic', tol=10,
             verbose=0)


# Final Model/Diagrams

In [ ]:
from sklearn.linear_model import ElasticNetCV
from sklearn.datasets import make_regression
import math 
from sklearn.metrics import max_error
from sklearn.metrics import mean_squared_error

import math 
from sklearn.metrics import mean_squared_error

In [ ]:


democrat = RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=50, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)


republican = RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [ ]:
democrat.fit(train_copy, y_train_democrat['percent'])
republican.fit(train_copy, y_train_republican['percent'])

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [ ]:
print("MAX ERROR")

print("\nDemocrat: \n")

y_pred_dem = democrat.predict(train_copy)
print("Train ME: {}".format((max_error(y_train_democrat['percent'], y_pred_dem))))

y_pred_test_dem = democrat.predict(test_copy)
print("Test ME: {}".format((max_error(y_test_democrat['percent'], y_pred_test_dem))))



####################
print("\nRepublican: \n")

y_pred_rep = republican.predict(train_copy)
print("Train ME: {}".format((max_error(y_train_republican['percent'], y_pred_rep))))

y_pred_test_rep = republican.predict(test_copy)
print("Test ME: {}".format((max_error(y_test_republican['percent'], y_pred_test_rep))))

print('-----------------------------------------------------------------------')

print("RMSE")

print("\nDemocrat: \n")

print("Train RMSE: {}".format(math.sqrt(mean_squared_error(y_train_democrat['percent'], y_pred_dem))))
print("Test RMSE: {}".format(math.sqrt(mean_squared_error(y_test_democrat['percent'], y_pred_test_dem))))


print("\nRepublican: \n")

print("Train RMSE: {}".format(math.sqrt(mean_squared_error(y_train_republican['percent'], y_pred_rep))))
print("Test RMSE: {}".format(math.sqrt(mean_squared_error(y_test_republican['percent'], y_pred_test_rep))))


MAX ERROR

Democrat: 

Train ME: 0.062049448917766536
Test ME: 0.5969909725130672

Republican: 

Train ME: 0.05573048135711964
Test ME: 0.5524381409958974
-----------------------------------------------------------------------
RMSE

Democrat: 

Train RMSE: 0.012630744729973479
Test RMSE: 0.1351540894584685

Republican: 

Train RMSE: 0.011932445536150565
Test RMSE: 0.13415718053771297


In [ ]:
print('-----------------------------------------------------------------------')
print("ACCURACY")

# 1: democrat, 0: republican
train_party = (y_train_democrat['percent'] > y_train_republican['percent']).astype(int)
test_party = (y_test_democrat['percent'] > y_test_republican['percent']).astype(int)

# datasets: y_pred_dem, y_pred_test_dem, y_pred_rep, y_pred_test_rep
train_pred = (y_pred_dem > y_pred_rep).astype(int)
test_pred = (y_pred_test_dem > y_pred_test_rep).astype(int)

print("train accuracy: {}".format(accuracy_score(train_party, train_pred)))
print("test accuracy: {}".format(accuracy_score(test_party, test_pred)))


-----------------------------------------------------------------------
ACCURACY
train accuracy: 1.0
test accuracy: 0.849624060150376


In [ ]:
colorscale = [
	'rgb(91,138,253)',
	'rgb(255,99,92)'
]

train_party[train_party == 1] = 'democrat'
train_party[train_party == 0] = 'republican'

fig = ff.create_choropleth(
	  fips = train_copy['fips'], values = train_party,
	  show_state_data = True, scope = ['VA'],
	  county_outline={'color': 'rgb(15,15,55)', 'width': 1},
	  state_outline={'color': 'rgb(15,15,55)', 'width': 1},
		colorscale = colorscale,
	  legend_title='Party ', 
    title='Majority vote in 2012 Election',
	  )

plotly.offline.iplot(fig)

test_party[test_party == 1] = 'democrat'
test_party[test_party == 0] = 'republican'

fig = ff.create_choropleth(
	  fips = test_copy['fips'], values = test_party,
	  show_state_data = True, scope = ['VA'],
	  county_outline={'color': 'rgb(15,15,55)', 'width': 1},
	  state_outline={'color': 'rgb(15,15,55)', 'width': 1},
		colorscale = colorscale,
	  legend_title='Party ', 
    title='Majority vote in 2016 Election',
	  )

plotly.offline.iplot(fig)

test_pred = test_pred.astype(str)
test_pred[test_pred == '1'] = 'democrat'
test_pred[test_pred == '0'] = 'republican'

fig = ff.create_choropleth(
	  fips = test_copy['fips'], values = test_pred,
	  show_state_data = True, scope = ['VA'],
	  county_outline={'color': 'rgb(15,15,55)', 'width': 1},
	  state_outline={'color': 'rgb(15,15,55)', 'width': 1},
		colorscale = colorscale,
	  legend_title='Party ', 
    title='Predicted majority vote in 2016 election',
	  )

plotly.offline.iplot(fig)



2020 predictions

In [ ]:
democrat_2020 = RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=50, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)


republican_2020 = RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)


democrat_2020.fit(test_copy, y_test_democrat['percent'])
republican_2020.fit(test_copy, y_test_republican['percent'])


y_pred_dem = democrat_2020.predict(test_copy)
y_pred_rep = republican_2020.predict(test_copy)
train_2020_party = y_pred_dem.copy().astype(str)
train_2020_party[y_pred_dem > y_pred_rep] = "democrat"
train_2020_party[y_pred_dem < y_pred_rep] = "republican"

print("Train Statistics")

print("Dem. RMSE: \t{}".format(math.sqrt(mean_squared_error(y_test_democrat['percent'], y_pred_dem))))
print("Repub. RMSE: \t{}".format(math.sqrt(mean_squared_error(y_test_republican['percent'], y_pred_rep))))
print("Accuracy: \t{}".format(accuracy_score(test_party, train_2020_party)))

dem_pred_2020 = democrat_2020.predict(data2020_copy)
rep_pred_2020 = republican_2020.predict(data2020_copy)

Train Statistics
Dem. RMSE: 	0.04826780274964606
Repub. RMSE: 	0.0468720170147562
Accuracy: 	0.9548872180451128


In [ ]:
endpts = [10, 20, 30, 40, 50, 60, 70]

fig = ff.create_choropleth(
	  fips = train_copy['fips'], values = dem_pred_2020*100,
	  show_state_data = True, scope = ['VA'],
	  county_outline={'color': 'rgb(15,15,55)', 'width': 1},
	  state_outline={'color': 'rgb(15,15,55)', 'width': 1},
		#colorscale = colorscale,
	  legend_title='Percent (%)', 
    title='Predicted Percent Vote for Democrat By County in 2020',
    binning_endpoints=endpts,
	  )

plotly.offline.iplot(fig)

fig = ff.create_choropleth(
	  fips = train_copy['fips'], values = rep_pred_2020*100,
	  show_state_data = True, scope = ['VA'],
	  county_outline={'color': 'rgb(15,15,55)', 'width': 1},
	  state_outline={'color': 'rgb(15,15,55)', 'width': 1},
		#colorscale = colorscale,
	  legend_title='Percent (%)', 
    title='Predicted Percent Vote for Republican By County in 2020',
    binning_endpoints=endpts,
	  )

plotly.offline.iplot(fig)

In [ ]:
plotly.offline.iplot(fig)

pred_2020 = test_pred.copy().astype(str)
pred_2020[dem_pred_2020 > rep_pred_2020] = 'democrat'
pred_2020[dem_pred_2020 < rep_pred_2020] = 'republican'

fig = ff.create_choropleth(
	  fips = test_copy['fips'], values = pred_2020,
	  show_state_data = True, scope = ['VA'],
	  county_outline={'color': 'rgb(15,15,55)', 'width': 1},
	  state_outline={'color': 'rgb(15,15,55)', 'width': 1},
		colorscale = colorscale,
	  legend_title='Party ', 
    title='Predicted majority vote in 2020 election',
	  )
